# Reading Data from Files, Writing Data to Files

So far we have learned about the different data types in python (integers, floats, booleans, strings), the different python data containers that can hold these data, whether those are native python containers (lists, tuples, dictionaries), or numpy arrays.  We will often encounter situations where we need to read data from a file into these containers, or write data in these containers to a file. This is a very common task in scientific computing that will  be covered in this notebook.

In Python, text is stored in strings in **text files**. We learned about the string (`str`) data type in the Week2 lecture. By text we mean sequences of alphanumeric characters that make sense to a human. In contrast, there are also so-called **binary files** used for large data outputs, or executable programs, that can only be read by a computer program.

There are several different ways to read and write text files in Python. We will cover the most common ones in turn which are: 
- Reading and writing files line-by-line
- Reading and writing files directly into arrays using the `numpy` library
- Reading and writing files using the `pandas` library

There are of course many other ways to read and write files in Python, but these are arguably the most common and useful ones. 

## The Data

The file `data/data_2MASS.txt` contains the magnitudes of some stars from the 2MASS astronomical sky survey.

``` text 
# Magnitudes of some stars from the 2MASS astronomical sky survey. 
# Coordinates are in decimal degrees in the J2000 equinox.
#  RA          DEC          Name          Jmag   e_Jmag
# (deg)       (deg)                       (mag)   (mag)
10.684737 +41.269035 J00424433+4116085   9.453  0.052
10.683469 +41.268585 J00424403+4116069   9.321  0.022
10.685657 +41.269550 J00424455+4116103  10.773  0.069
10.686026 +41.269226 J00424464+4116092   9.299  0.063
10.683465 +41.269676 J00424403+4116108  11.507  0.056
10.686015 +41.269630 J00424464+4116106   9.399  0.045
10.685270 +41.267124 J00424446+4116016  12.070  0.035
```

The lines that start with `#` are comments that make up the file **header**. A header provides descriptive information or metadata (like units) about what is stored in the file and how it is arranged, but does not constitute the data itself. Then we see a table of data with 5 columns. The first two columns contain floating point numbers (floats) which are  the coordinates (right ascension and declination) of the star in decimal degrees. The third column is a string, which is name of the star.  The fourth and fifth columns also contain floats which represent the $J$-band magnitude of the star (`Jmag`), and the error on this measurement (`e_Jmag`). In Astronomy, the magnitude of a star is a logarithmic measure of its brightness (the $\log_{10}$ of its intensity), and $J$ represents the astronomical filter that was used for the measurements. 

## Reading Data from Files Line-by-Line

We have already talked about several built-in Python data types (lists, tuples, dictionaries), but there are other types that we did not discuss. One of these is the file() object which can be used to read or write files. Let's try and access the contents of the data file in Python. We start off by creating a file object:

**Note if you are running this notebook on Google Colab**, you will need to download the data files first onto the Google drive file system, create the data and output directories and move the files to the data directory.  You can do this by running the following code snippet in a code cell:

``` python
import urllib.request
url = "https://raw.githubusercontent.com/enigma-igm/Phys29/main/Phys29/lectures/Week4/data/data_2MASS.txt"
urllib.request.urlretrieve(url, "data_2MASS.txt")
url = "https://raw.githubusercontent.com/enigma-igm/Phys29/main/Phys29/lectures/Week4/data/data_2MASS.csv"
urllib.request.urlretrieve(url,  "data_2MASS.csv")
!mkdir -p data
!mv -f data_2MASS.txt data
!mv -f data_2MASS.csv data
!mkdir -p output
```

In [ ]:
f2MASS = open('data/data_2MASS.txt', 'r')

The open function is taking the `data/data_2MASS.txt` file, opening it, and returning an object (which we call `f2MASS`) that can then be used to access the file contents. 

Note that f2MASS is not the data in the file, it is what is called a file handle, which points to the file:

In [ ]:
type(f2MASS)

Now we simply type:

In [ ]:
f2MASS.read()

The read() function just read the whole file and put the contents inside a giant string. Let's try this again: 

In [ ]:
f2MASS.read()

What happened? The output is an empty string `''`. We read the file, and the file **pointer** is now sitting at the end of the file, and hence there is nothing left to read. Let's now and try to do something more useful and capture the contents of the file in a string: 

In [ ]:
f2MASS = open('data/data_2MASS.txt', 'r')
string_data = f2MASS.read()
f2MASS.close()

Above we opened the file, read the contents, and then closed the file. We can examine the contents of `string_data`

In [ ]:
string_data

If we instead use the print() function we get something that looks more like our file contents: 

In [ ]:
print(string_data)

Note that `string_data` is one giant string containing all the data in the file. 

In [ ]:
len(string_data)

The different lines are separated by the special character `\n`, or the **newline** character, which is used to indicate the end of a line. The reason why the native Python print() function prints the contents of the file in a nice way is because it interprets the `\n` character and prints a new line whenever it encounters it. 

This form of the data is not very useful. We want to access the data line-by-line.  There are multiple ways to do this, but the simplest and most common is to use a for loop. 

In [ ]:
with open('data/data_2MASS.txt', 'r') as f2MASS:
    for line in f2MASS:
        print(repr(line))

First, what does the `with` statement do?  When you're reading a file in Python, you can use the `with` statement to handle the file. The `with` statement makes sure the file is properly opened before starting the block of code that follows, and then correctly closed after that block of code is done. After the code block executes that reads the file line-by-line it is no longer necessary to have the file open.  Whereas above we explicitly closed the file using the `f2MASS.close()` function when we were done, the `with` statement closes the file for us automatically.  By closing the file after the `with` block is completed, the code is easier to read. This also helps prevent bugs, like forgetting to close the file when you're done with it, which can lead to a program not exiting properly. 

Second, note that in our foor loop we are now looping over a file rather than a list, and this automatically reads in the next line at each iteration. In other words, the Python file object is an **iterator**.  Each line is being returned as a string as we execute the loop. When we print the string, we are using `repr()` to show any invisible characters. Notice the `\n` newline character indicating the end of each line.

Now we're reading in a file line by line, what would be nice would be to get some values out of it.  Let's examine the last line in detail. If we just type `line` we should see the last line that was printed in the loop:

In [ ]:
line

We can use the `split()` function to split the line into a list of strings. By default, the `split()` function splits on whitespace (spaces, tabs, newlines) so we don't need to worry about the exact number of spaces or the newline character `\n` at the end of the line (there is a method called `strip()` that can be used to remove the newline character if needed, but we don't need that here).

In [ ]:
columns = line.split()
columns

Finally, if we want to access the data in the different columns, we can use the list indexing that we learned about in Week3. For example, 


In [ ]:
this_ra = columns[0]
this_dec = columns[1]
this_name = columns[2]
this_Jmag = columns[3]
this_eJmag = columns[4]
# Or equivalently, we could write
# this_ra, this_dec, this_name, this_Jmag, this_eJmag = columns

In [ ]:
this_name

In [ ]:
this_Jmag

Note that whereas this_name is a string, this_Jmag is actually a floating point number.  But because the file lines are stored as text, and read in as strings, and the `split()` function returns a list of strings, all of the data that we have just read in and split are strings.  We need to convert the strings to numbers if we want to do any calculations with them.  This can be done by casting the strings into floats using native Python `float()` function:

In [ ]:
this_jmag = float(columns[3])
this_jmag

So you can see now that this_jmag is a floating point number rather than a string. 

Putting everything we have learned about files, reading data, basic control flow, loops and numpy arrays together, we can now write a program that reads in the data from the file, converts them to the proper data types, stores them in numpy arrays, and then prints out the contents of the arrays to the screen:

In [ ]:
import numpy as np

# Initialize empty lists
ra_list = []
dec_list = []
name_list = []
jmag_list = []
e_jmag_list = []

# Open the file
with open('data/data_2MASS.txt', 'r') as file:
    # Read and parse the lines
    for line in file:
        # Skip comment lines
        if line.startswith('#'):
            continue

        # Split the line into columns
        columns = line.split()

        # Parse the columns and append to the lists
        ra_list.append(float(columns[0]))
        dec_list.append(float(columns[1]))
        name_list.append(columns[2])
        jmag_list.append(float(columns[3]))
        e_jmag_list.append(float(columns[4]))

# Convert the lists to arrays
ra = np.array(ra_list)
dec = np.array(dec_list)
names = np.array(name_list)
jmag = np.array(jmag_list)
e_jmag = np.array(e_jmag_list)

# Print the arrays
print('RA:', ra)
print('DEC:', dec)
print('Name:', name_list)
print('Jmag:', jmag)
print('e_Jmag:', e_jmag)

## Writing Data to Files Line-by-Line

Now that we have read the data from a file, we might want to write the data to a new file. To open a file for writing we write:

In [ ]:
foutput = open('output/write_test.txt', 'w')

Then simply use the `write()` function to write any content to the file, for example:

In [ ]:
foutput.write("Hello, World!\n")

If you want to write multiple lines, you can either give a list of strings to the `writelines()` method:

In [ ]:
foutput.writelines(['spam\n', 'egg\n', 'foo\n'])

or you can write them as a single string:

In [ ]:
foutput.write('spam\negg\nfoo\n')

Once you have finished writing data to a file, you need to close it:

In [ ]:
foutput.close()

We can now inspect the contents of the file we created using the terminal `cat` command:

In [ ]:
!cat output/write_test.txt

Similar to when we put the block of code that dealt with reading data from a file behind the `with` statement, it is also conventional to put the writing code block behind a `with` statement.  
As we have seen, files must not just be opened but should be properly closed afterwards to make sure they are actually written before using them somewhere else. Sometimes writes to files get cached by Python to minimize actual writing to disk, which is comparably slow. Closing a file ensures that these changes are actually written to disk. 

Putting together what we have learned about writing files, we can now write out the contents of the arrays that we created above to a new file:

In [ ]:
# Open the file in write mode
with open('output/output_2MASS.txt', 'w') as file:
    # Write the header lines
    file.write("# Magnitudes of some stars from the 2MASS astronomical sky survey.\n")
    file.write("# Coordinates are in decimal degrees in the J2000 equinox.\n")
    file.write("#  RA         DEC         Name               Jmag   e_Jmag\n")
    file.write("# (deg)      (deg)                           (mag)   (mag)\n")

    # Write the data line by line
    for i in range(len(ra)):
        line = "{:10.6f} {:10.6f} {:<20s} {:7.3f}  {:.3f}\n".format(ra[i], dec[i], names[i], jmag[i], e_jmag[i])
        file.write(line)


Note that the `with` statement opens the file and assigns it to the variable `file`, then we use the `write()` function to write the four lines of the file header. Finally, a for loop is used to write the contents of the arrays to the file line-by-line.  Note that we have explicitly specified the output format of each line of the output file, specifically the number of decimal places to use for the floating point numbers and the number of characters to write out for the strings.  There are a few different ways to specify the output format of data in python (see [here](https://docs.python.org/3/tutorial/inputoutput.html#fancier-output-formatting) for more details).  

## Read and Writing Data Using Pandas

Pandas is a Python package that provides high-performance and easy to use data structures and data analysis tools. Specifically, pandas has useful utilities for reading and writing data from and to files. Below we provide examples of reading and writing data using pandas for the same data from the `data_2MASS.txt` file as above. Then we show how to use pandas to read and write data from and to a CSV file, which is a common file format for storing tabular data that has some advantages over the plain text files that we focused on for most of this lecture.

To read in the data from the `data_2MASS.txt` file using pandas, we use the `read_csv()` function:

In [1]:
import pandas as pd

# Use pd.read_csv to read the data from the file
data = pd.read_csv('data/data_2MASS.txt', sep=r'\s+', comment='#', 
                   names=['RA', 'DEC', 'Name', 'Jmag', 'e_Jmag'])

# Extract the columns and convert to numpy arrays
ra = data['RA'].values
dec = data['DEC'].values
names = data['Name'].values
jmag = data['Jmag'].values
e_jmag = data['e_Jmag'].values

# Print the arrays
print('RA:', ra)
print('DEC:', dec)
print('Name:', names)
print('Jmag:', jmag)
print('e_Jmag:', e_jmag)

RA: [10.684737 10.683469 10.685657 10.686026 10.683465 10.686015 10.68527 ]
DEC: [41.269035 41.268585 41.26955  41.269226 41.269676 41.26963  41.267124]
Name: ['J00424433+4116085' 'J00424403+4116069' 'J00424455+4116103'
 'J00424464+4116092' 'J00424403+4116108' 'J00424464+4116106'
 'J00424446+4116016']
Jmag: [ 9.453  9.321 10.773  9.299 11.507  9.399 12.07 ]
e_Jmag: [0.052 0.022 0.069 0.063 0.056 0.045 0.035]


In the name of the reading function `read_csv`, the abbreviation **CSV** stands for **comma-separated values**.  Wheras in our file `data_2MASS.txt` we separated the columns of data with spaces, in a CSV file the data entries are separated by commas.  To tell `pandas` that our data are separated with whitespaces of any length, we used the `sep=r'\s+'` syntax, which is a **regular expression** that matches any number of whitespace characters (hence the `+`). We also used the `comment=#` syntax to tell `pandas` to ignore our header lines that start with the `#` character.  Finally, we input the names of the various columns in our data file using the `names=` syntax. 

The `read_csv()` function returns a pandas `DataFrame` object.  A `DataFrame`` is a 2-dimensional (rows $\times$ columns) labeled data structure with columns of potentially different types. You can think of it like a spreadsheet, and it is the most commonly used pandas object. 

In [2]:
data?

Type:        DataFrame
String form:
RA        DEC               Name    Jmag  e_Jmag
           0  10.684737  41.269035  J00424433+411 <...> 630  J00424464+4116106   9.399   0.045
           6  10.685270  41.267124  J00424446+4116016  12.070   0.035
Length:      7
File:        ~/miniconda3/envs/Phys29/lib/python3.12/site-packages/pandas/core/frame.py
Docstring:  
Two-dimensional, size-mutable, potentially heterogeneous tabular data.

Data structure also contains labeled axes (rows and columns).
Arithmetic operations align on both row and column labels. Can be
thought of as a dict-like container for Series objects. The primary
pandas data structure.

Parameters
----------
data : ndarray (structured or homogeneous), Iterable, dict, or DataFrame
    Dict can contain Series, arrays, constants, dataclass or list-like objects. If
    data is a dict, column order follows insertion-order. If a dict contains Series
    which have an index defined, it is aligned by its index. This alignment also

One nice thing about pandas `DataFrames` is that they display nicely in Jupyter notebooks.  Let's take a look at the DataFrame that we just created:

In [3]:
data

,RA,DEC,Name,Jmag,e_Jmag
0,10.684737,41.269035,J00424433+4116085,9.453,0.052
1,10.683469,41.268585,J00424403+4116069,9.321,0.022
2,10.685657,41.269550,J00424455+4116103,10.773,0.069
3,10.686026,41.269226,J00424464+4116092,9.299,0.063
4,10.683465,41.269676,J00424403+4116108,11.507,0.056
5,10.686015,41.269630,J00424464+4116106,9.399,0.045
6,10.685270,41.267124,J00424446+4116016,12.070,0.035


It is interesting to note that whereas in our line-by-line file reading example above we had to explicitly convert the strings to floating point numbers, pandas has automatically done this for us.  This is because pandas is *smart* enough to recognize that the columns of data in our file are floating point numbers.  We can check the data types of the columns in our DataFrame using the `dtype` attribute: 

In [4]:
data.dtypes

RA        float64
DEC       float64
Name       object
Jmag      float64
e_Jmag    float64
dtype: object

The string names of the stars are stored in the colum `Name` and have data type `object`, which is the pandas data type for strings.  The rest of the data columns have data type `float64`, which we expect. 

As you can see `pandas` can read the data in our file `data_2MASS.txt` into a `DataFrame` object, but we had to provide it with a lot of guidance via the keyword arguments to `read_csv`. An alternative would be to use a more standarized file format for storing the data, namely the CSV standard. 

The file `data_2MASS.csv` contains the same data as the `data_2MASS.txt` file, but in CSV format. It looks like this:

``` text
# Magnitudes of some stars from the 2MASS astronomical sky survey. 
# Coordinates are in decimal degrees in the J2000 equinox.
# Units: RA (deg), DEC (deg), Name, Jmag (mag), e_Jmag (mag)
RA,DEC,Name,Jmag,e_Jmag
10.684737,+41.269035,00424433+4116085,9.453,0.052
10.683469,+41.268585,00424403+4116069,9.321,0.022
10.685657,+41.269550,00424455+4116103,10.773,0.069
10.686026,+41.269226,00424464+4116092,9.299,0.063
10.683465,+41.269676,00424403+4116108,11.507,0.056
10.686015,+41.269630,00424464+4116106,9.399,0.045
10.685270,+41.267124,00424446+4116016,12.070,0.035
```

Note that the first (non-comment) line of the file contains the names of the columns, and the data entries are separated by commas.  This is the standard format for a CSV file.

We can now read in the data from the `data_2MASS.csv` file using the `pandas` `read_csv()` function. Notice the syntax is quite a bit simpler now:

In [5]:
# Use pd.read_csv to read the data from the file
data = pd.read_csv('data/data_2MASS.csv', comment='#')
data.dtypes

RA        float64
DEC       float64
Name       object
Jmag      float64
e_Jmag    float64
dtype: object

In [6]:
# Assign the pandas columns to numpy arrays as before
ra = data['RA'].values
dec = data['DEC'].values
names = data['Name'].values
jmag = data['Jmag'].values
e_jmag = data['e_Jmag'].values

# Print the arrays
print('RA:', ra)
print('DEC:', dec)
print('Name:', names)
print('Jmag:', jmag)
print('e_Jmag:', e_jmag)

RA: [10.684737 10.683469 10.685657 10.686026 10.683465 10.686015 10.68527 ]
DEC: [41.269035 41.268585 41.26955  41.269226 41.269676 41.26963  41.267124]
Name: ['00424433+4116085' '00424403+4116069' '00424455+4116103'
 '00424464+4116092' '00424403+4116108' '00424464+4116106'
 '00424446+4116016']
Jmag: [ 9.453  9.321 10.773  9.299 11.507  9.399 12.07 ]
e_Jmag: [0.052 0.022 0.069 0.063 0.056 0.045 0.035]


Now what about writing data to a file using Pandas?

In [7]:
# Create a Pandas DataFrame from our numpy arrays
df = pd.DataFrame({
    'RA': ra,
    'DEC': dec,
    'Name': names,
    'Jmag': jmag,
    'e_Jmag': e_jmag
})

# Define the header
header = ("# Magnitudes of some stars from the 2MASS astronomical sky survey.\n"
          "# Coordinates are in decimal degrees in the J2000 equinox.\n"
          "#  RA         DEC         Name               Jmag   e_Jmag\n"
          "# (deg)      (deg)                           (mag)   (mag)\n")

# Write the header to the file. Pandas does not support writing these # delimited headers, so we have to do it manually.
with open('output/output_with_header.csv', 'w') as f:
    f.write(header)
# Write the DataFrame to a file. Notice that we use the mode='a' option to append to the file, 
# since we already created a new file for writing when we wrote the header
df.to_csv('output/output_with_header.csv', mode='a', index=False, float_format="%.6f")

The CSV format default is to write out an extra column at the beginning indicating the index of each line. Since we just wanted the data values, we suppressed this with `index=False`. Alternativey, without writing out the header information, writing a CSV file is even simpler:

In [8]:
# Write the DataFrame to a file
df.to_csv('output/output_no_header.csv', index=False, float_format="%.6f")

We can now check that we wrote a valid CSV file by reading it back in using `pandas`:

In [9]:
df_with_header = pd.read_csv('output/output_with_header.csv', comment='#')

In [10]:
df_no_header = pd.read_csv('output/output_no_header.csv')

## Read and Writing Data Using NumPy

The `numpy` package also has methods to read and write data. We provide examples of reading using `np.loadtxt` and writing using `np.savetxt` using the same data from the `data_2MASS.txt` file as above.

In [ ]:
import numpy as np

# Use np.loadtxt to read the data from the file
data = np.loadtxt('data/data_2MASS.txt', dtype={'names': ('RA', 'DEC', 'Name', 'Jmag', 'e_Jmag'),
                                           'formats': ('f8', 'f8', 'S20', 'f8', 'f8')})

# Extract the columns
ra = data['RA']
dec = data['DEC']
names = data['Name'].astype(str)
jmag = data['Jmag']
e_jmag = data['e_Jmag']

# Print the arrays
print('RA:', ra)
print('DEC:', dec)
print('Name:', names)
print('Jmag:', jmag)
print('e_Jmag:', e_jmag)

In [ ]:
# Create a structured array with your data. We do this becuase we have data with different types. 
# Another option would have been to cast everything to dtype=str (U20) and use np.column_stack
data = np.zeros(len(ra), dtype={'names':('RA', 'DEC', 'Name', 'Jmag', 'e_Jmag'),
                                 'formats':('f8', 'f8', 'U20', 'f8', 'f8')})

data['RA'] = ra
data['DEC'] = dec
data['Name'] = names
data['Jmag'] = jmag
data['e_Jmag'] = e_jmag

# Define the header and footer
header = ("# Magnitudes of some stars from the 2MASS astronomical sky survey.\n"
          "# Coordinates are in decimal degrees in the J2000 equinox.\n"
          "#  RA         DEC      Name                  Jmag     e_Jmag\n"
          "# (deg)      (deg)                           (mag)    (mag)")

# Define the format for each field
formats = "%10.6f %10.6f %-20s %7.3f  %7.3f"

# Write the data to the file
np.savetxt('output/output_numpy.txt', data, fmt=formats, header=header, comments='')